In [ ]:
import os
import shutil
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import torch
from ultralytics import YOLO
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from PIL import Image

In [ ]:
# Define dataset paths
ROOT_DIR = '/content/drive/MyDrive/DATASET'
TRAIN_DIR = os.path.join(ROOT_DIR, 'train')
VAL_DIR = os.path.join(ROOT_DIR, 'val')
TEST_DIR = os.path.join(ROOT_DIR, 'test')

In [ ]:
# Validate dataset
print("Checking dataset...")
if not all(os.path.exists(path) for path in [TRAIN_DIR, VAL_DIR, TEST_DIR]):
    raise FileNotFoundError("One or more dataset directories are missing!")
print("Dataset found!")

In [ ]:
# Load YOLO model
model = YOLO('yolov8l-oiv7.pt')

In [ ]:
# Train model
results = model.train(
    data='/content/drive/MyDrive/DATASET/data.yaml',
    epochs=30,
    imgsz=640,
    batch=8,
    workers=4,
    lr0=0.01,
    optimizer='SGD',
    momentum=0.937,
    weight_decay=0.0005,
)


In [ ]:
# Save trained model to Google Drive
shutil.copytree('runs', '/content/drive/MyDrive/DATASET/runs')

# Load training results
df = pd.read_csv('/content/runs/detect/train2/results.csv')
df.columns = df.columns.str.strip()

In [ ]:
# Plot training metrics
fig, axs = plt.subplots(nrows=5, ncols=2, figsize=(15, 15))
sns.lineplot(x='epoch', y='train/box_loss', data=df, ax=axs[0,0])
sns.lineplot(x='epoch', y='train/cls_loss', data=df, ax=axs[0,1])
sns.lineplot(x='epoch', y='train/dfl_loss', data=df, ax=axs[1,0])
sns.lineplot(x='epoch', y='metrics/precision(B)', data=df, ax=axs[1,1])
sns.lineplot(x='epoch', y='metrics/recall(B)', data=df, ax=axs[2,0])
sns.lineplot(x='epoch', y='metrics/mAP50(B)', data=df, ax=axs[2,1])
sns.lineplot(x='epoch', y='metrics/mAP50-95(B)', data=df, ax=axs[3,0])
sns.lineplot(x='epoch', y='val/box_loss', data=df, ax=axs[3,1])
sns.lineplot(x='epoch', y='val/cls_loss', data=df, ax=axs[4,0])
sns.lineplot(x='epoch', y='val/dfl_loss', data=df, ax=axs[4,1])
plt.suptitle('Training Metrics and Loss', fontsize=24)
plt.subplots_adjust(top=0.8)
plt.tight_layout()
plt.show()

In [ ]:
# Load best trained model
model = YOLO('runs/detect/train2/weights/best.pt')

In [ ]:
# Validation Metrics
metrics = model.val(data='/content/drive/MyDrive/DATASET/data.yaml', conf=0.25, iou=0.45, task='val')
print("mAP50-95:", metrics.box.map)
print("mAP50:", metrics.box.map50)
print("mAP75:", metrics.box.map75)

In [ ]:
# Load and display confusion matrix
image_path = '/content/runs/detect/train2/confusion_matrix.png'
image = Image.open(image_path)
plt.figure(figsize=(12, 12))
plt.imshow(image)
plt.axis('off')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Perform inference on test set
test_dataset = '/content/drive/MyDrive/DATASET/test/images'
results = model.predict(source=test_dataset, save=True)

In [ ]:
# Extract ground-truth vs predicted labels
true_labels, pred_labels = [], []
for result in results:
    for box in result.boxes:
        true_labels.append(box.cls.numpy())
        pred_labels.append(box.conf.numpy())

In [ ]:
# Classification Report
print("\nClassification Report:")
print(classification_report(true_labels, pred_labels))

# Confusion Matrix
conf_matrix = confusion_matrix(true_labels, pred_labels)
conf_matrix_normalized = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]
sns.heatmap(conf_matrix_normalized, annot=True, fmt='.2f', cmap='Blues')
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Normalized Confusion Matrix")
plt.show()